<center> <img src="https://oci02.img.iteso.mx/Identidades-De-Instancia/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg" align="center" width="650" height="400"/> </center>

<center><font color=#555555><font size=6> MAF1731 - Microstructure and Trading Systems </font> <br> <br>
    
<center><font color=#555555><font size=5> B.Eng Financial Engineering </font> <br> <br> <br> <br>

<center><font color=#555555><font size=6> I.F. Gerardo González Flores</font> <br> <br>
    
<center><font color=#555555><font size=6> I.F. Tania Bell Nicholls</font> <br> <br>

<center> <font color= #555555> <font size = 4> April 2023 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/Tania-Bell/MyST_Lab4'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Lab 4 </font> <br> <br> <font color= #555555> <font size = 5>Market's Microstructure </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

<font color= #555555> <font size = 4> Abstract </font>

<font color= #6B6B6B> <font size = 3>  Throughout this laboratory, we will use the information from the Cryptocurrency's Book of Orders from various international exchanges, with the primary objective of visualizing and modeling the market's microstructure through it's characteristics and behaviors like Matchine Engine, Levels, Ask-Bid Volumes, etc.    </font>

<br>

# <font color= #6B6B6B> 0. Install/Load Packages and Depedencies </font>

<hr style="border:0.02in solid gray"> </hr>

## <font color= #6B6B6B> 0.2 Python Packages </font> <font color= #555555>

In order to run this notebook, it is necessary to have installed and/or have the _requirements.txt_ file with the following: 

- pandas
- numpy
- jupyter
- ccxt
- ast
- time
- asyncio

## <font color= #6B6B6B> 0.3 Files Dependencies </font> <font color= #555555>

The following are the file dependencies that are needed to run this notebook:

Because of the incompatability of the jupyter versions the book of orders were saved as csv.

- binance_BTCUSDT.csv (referred to as Exchange 1)
- binance_ETHUSDT.csv (Exchange 2)
- bitfinex_BTCUSDT.csv (Exchange 3)
- bitfinex_ETHUSDT.csv (Exchange 4)
- huobi_BTCUSDT.csv (Exchange 5)
- huobi_ETHUSDT.csv (Exchange 6)

## <font color= #6E6E6E> 0.4 Install Packages</font>

In [1]:
%%capture

# Install all the pip packages in the requirements.txt
import sys
!{sys.executable} -m pip install -r requirements.txt

In [2]:
# importing all libraries
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
import numpy as np
import pandas as pd
import ccxt
import functions as fn
import data as dt
import visualizations as vn
import plotly.express as px

# <font color= #6B6B6B> <font size = 5> 1. Introduction </font>

This laboratory focuses on using the information in the cryptocurrencies' Book of Orders to visualize and model the market's microstructure. In order to learn about a market's microstructure it is important to have all the information available, for that reason the market chosen for this project was the cryptocurrecy.

This project starts with obtaining the most accurate and current information of three different exchange markets and three different trading pairs using the ccxt library. The exchange markets chosen were Binance and Bitfinex. The trading pairs chosen were BTC/USDT, ETH/USDT and ADA/USDT. Then the information was organized and summarized taking into account the levels and total bid size and ask size.

For the visualization, graphs of the mid price were generated with the plotly library in order to compare the different time series.

To complete the objetives, the last part of the lab consisted of modeling the microstructure of the market. This was achieved through estimating the Effective Spread based on the Roll model (1984).

# <font color= #6B6B6B> <font size = 5> 2. Objectives </font>

- Obtain, clean and analyze the information obtained from the Book of Orders
- Visualize the mid price time series
- Model the market's microstructure using the effective spread

# <font color= #6B6B6B> <font size = 5> 3. Data Description </font>

<hr style="border:0.02in solid gray"> </hr>

Exchange markets:
- Binance
- Bitfinex
- Huobi

Trading pairs:
- BTC/USDT
- ETH/USDT

Datetime : timestamp in BO

Ask_size : volume of asks

Ask_price : price of asks

Bid_size : volume of bids

Bid_price : price of bids

Spread : difference between ask and bid

Mid_price : average Top of the Book

VWAP : volume-weighted average price

Close_price : closing price

Levels : trading levels

### Data

The data obtained from the ccxt library of the Book of Orders is displayed below:

#### Exchange 1

In [3]:
data_files = dt.get_file_names("files")
pd.read_csv("files/"+data_files[0])

,Unnamed: 0,exchange,datetime,orderbook
0,0,binance,2023-04-22T00:31:03.106Z,"{'ask_size': [2.94389, 0.0063, 2.40838, 0.0006..."
1,1,binance,2023-04-22T00:31:03.623Z,"{'ask_size': [3.01702, 0.0063, 2.40838, 0.0006..."
2,2,binance,2023-04-22T00:31:04.456Z,"{'ask_size': [3.01219, 0.0063, 2.40838, 0.0006..."
3,3,binance,2023-04-22T00:31:05.474Z,"{'ask_size': [2.95339, 0.0063, 2.40838, 0.0006..."
4,4,binance,2023-04-22T00:31:06.830Z,"{'ask_size': [3.29879, 0.0063, 2.40838, 0.0006..."
5,5,binance,2023-04-22T00:31:07.794Z,"{'ask_size': [2.11278, 0.01629, 0.00682, 0.000..."
6,6,binance,2023-04-22T00:31:08.295Z,"{'ask_size': [7.87288, 0.01629, 0.00682, 0.000..."
7,7,binance,2023-04-22T00:31:09.166Z,"{'ask_size': [7.45218, 0.01629, 0.00682, 0.000..."
8,8,binance,2023-04-22T00:31:10.084Z,"{'ask_size': [7.85656, 0.01629, 0.00682, 0.000..."
9,9,binance,2023-04-22T00:31:11.001Z,"{'ask_size': [7.38832, 0.01629, 0.00682, 0.000..."


#### Exchange 2

In [4]:
pd.read_csv("files/"+data_files[1])

,Unnamed: 0,exchange,datetime,orderbook
0,0,binance,2023-04-22T00:32:26.508Z,"{'ask_size': [64.0842, 0.0087, 3.7989, 8.6555,..."
1,1,binance,2023-04-22T00:32:27.315Z,"{'ask_size': [57.362, 0.0087, 3.7989, 8.6555, ..."
2,2,binance,2023-04-22T00:32:28.379Z,"{'ask_size': [62.7687, 0.0087, 3.7989, 8.6555,..."
3,3,binance,2023-04-22T00:32:29.428Z,"{'ask_size': [65.462, 0.0087, 3.7989, 8.6555, ..."
4,4,binance,2023-04-22T00:32:30.451Z,"{'ask_size': [64.9541, 0.0128, 3.7989, 8.6555,..."
5,5,binance,2023-04-22T00:32:31.836Z,"{'ask_size': [64.9541, 0.0128, 3.7989, 8.6555,..."
6,6,binance,2023-04-22T00:32:32.807Z,"{'ask_size': [70.3613, 0.0128, 3.7989, 8.6555,..."
7,7,binance,2023-04-22T00:32:34.116Z,"{'ask_size': [63.5765, 0.0128, 3.7989, 8.6555,..."
8,8,binance,2023-04-22T00:32:34.692Z,"{'ask_size': [63.4186, 0.0128, 3.7989, 8.6555,..."
9,9,binance,2023-04-22T00:32:35.300Z,"{'ask_size': [63.2546, 0.0128, 3.7989, 8.6555,..."


#### Exchange 3

In [5]:
pd.read_csv("files/"+data_files[2])

,Unnamed: 0,exchange,datetime,orderbook
0,0,bitfinex,2023-04-24T22:31:28.293Z,"{'ask_size': [0.207516, 0.0878, 0.1275, 0.1291..."
1,1,bitfinex,2023-04-24T22:31:29.874Z,"{'ask_size': [0.207516, 0.0878, 0.1275, 0.1291..."
2,2,bitfinex,2023-04-24T22:31:31.858Z,"{'ask_size': [0.02, 0.00197795, 0.22751, 0.215..."
3,3,bitfinex,2023-04-24T22:31:33.409Z,"{'ask_size': [0.02, 0.00197795, 0.22751, 0.215..."
4,4,bitfinex,2023-04-24T22:31:34.960Z,"{'ask_size': [0.02, 0.00197795, 0.22751, 0.215..."
5,5,bitfinex,2023-04-24T22:31:36.719Z,"{'ask_size': [0.147499, 0.05, 0.12929391, 0.78..."
6,6,bitfinex,2023-04-24T22:31:38.185Z,"{'ask_size': [0.147499, 0.05, 0.12929391, 0.78..."
7,7,bitfinex,2023-04-24T22:31:39.874Z,"{'ask_size': [0.147499, 0.05, 0.12929391, 0.78..."
8,8,bitfinex,2023-04-24T22:31:41.519Z,"{'ask_size': [0.147499, 0.127492, 0.15, 0.1274..."
9,9,bitfinex,2023-04-24T22:31:43.042Z,"{'ask_size': [0.147499, 0.127492, 0.15, 0.1274..."


#### Exchange 4

In [6]:
pd.read_csv("files/"+data_files[3])

,Unnamed: 0,exchange,datetime,orderbook
0,0,bitfinex,2023-04-24T23:39:32.853Z,"{'ask_size': [3.79657, 0.0174057, 5.29616, 3.3..."
1,1,bitfinex,2023-04-24T23:39:34.467Z,"{'ask_size': [3.79657, 0.0174057, 5.29616, 3.3..."
2,2,bitfinex,2023-04-24T23:39:36.299Z,"{'ask_size': [3.7964, 5.29616, 4.278, 3.89207,..."
3,3,bitfinex,2023-04-24T23:39:37.949Z,"{'ask_size': [3.7964, 5.29616, 4.278, 3.89207,..."
4,4,bitfinex,2023-04-24T23:39:39.475Z,"{'ask_size': [3.7964, 5.29616, 4.278, 3.89207,..."
5,5,bitfinex,2023-04-24T23:39:41.495Z,"{'ask_size': [0.01909068, 3.79452, 2.17195053,..."
6,6,bitfinex,2023-04-24T23:39:43.068Z,"{'ask_size': [0.01909068, 3.79452, 2.17195053,..."
7,7,bitfinex,2023-04-24T23:39:45.004Z,"{'ask_size': [0.02000633, 0.33785053, 5.29429,..."
8,8,bitfinex,2023-04-24T23:39:46.501Z,"{'ask_size': [0.02000633, 0.33785053, 5.29429,..."
9,9,bitfinex,2023-04-24T23:39:48.004Z,"{'ask_size': [0.02000633, 0.33785053, 5.29429,..."


#### Exhange 5

In [7]:
pd.read_csv("files/"+data_files[4])

,Unnamed: 0,exchange,datetime,orderbook
0,0,huobi,2023-04-25T01:15:45.453Z,"{'ask_size': [0.3652, 0.036414, 0.254864, 0.14..."
1,1,huobi,2023-04-25T01:15:46.289Z,"{'ask_size': [0.3652, 0.036414, 0.254864, 0.23..."
2,2,huobi,2023-04-25T01:15:47.139Z,"{'ask_size': [0.3652, 0.036414, 0.254864, 0.23..."
3,3,huobi,2023-04-25T01:15:47.671Z,"{'ask_size': [0.3652, 0.036414, 0.254864, 0.23..."
4,4,huobi,2023-04-25T01:15:48.221Z,"{'ask_size': [0.3652, 0.036414, 0.1, 0.254864,..."
5,5,huobi,2023-04-25T01:15:48.742Z,"{'ask_size': [0.3652, 0.036414, 0.1, 0.254864,..."
6,6,huobi,2023-04-25T01:15:49.264Z,"{'ask_size': [0.3652, 0.036414, 0.1, 0.254864,..."
7,7,huobi,2023-04-25T01:15:49.790Z,"{'ask_size': [0.3652, 0.036415, 0.1, 0.254864,..."
8,8,huobi,2023-04-25T01:15:50.329Z,"{'ask_size': [0.3652, 0.036415, 0.1, 0.254864,..."
9,9,huobi,2023-04-25T01:15:50.883Z,"{'ask_size': [0.3652, 0.036415, 0.1, 0.254864,..."


#### Exchange 6

In [8]:
pd.read_csv("files/"+data_files[5])

,Unnamed: 0,exchange,datetime,orderbook
0,0,huobi,2023-04-25T01:21:29.886Z,"{'ask_size': [1.0083, 8.6181, 0.2742, 0.0298, ..."
1,1,huobi,2023-04-25T01:21:30.790Z,"{'ask_size': [1.0083, 8.6181, 0.2742, 0.0298, ..."
2,2,huobi,2023-04-25T01:21:31.482Z,"{'ask_size': [1.0083, 8.6181, 0.2742, 0.0298, ..."
3,3,huobi,2023-04-25T01:21:32.299Z,"{'ask_size': [1.0075, 8.6181, 0.2742, 0.0298, ..."
4,4,huobi,2023-04-25T01:21:32.836Z,"{'ask_size': [1.0083, 8.6181, 0.2742, 0.0298, ..."
5,5,huobi,2023-04-25T01:21:33.364Z,"{'ask_size': [1.0075, 8.6181, 0.2742, 0.0298, ..."
6,6,huobi,2023-04-25T01:21:34.033Z,"{'ask_size': [1.0075, 8.6181, 0.2742, 0.0298, ..."
7,7,huobi,2023-04-25T01:21:34.620Z,"{'ask_size': [0.9999, 8.6181, 0.2742, 0.0298, ..."
8,8,huobi,2023-04-25T01:21:35.260Z,"{'ask_size': [1.0075, 8.6181, 0.2742, 0.0298, ..."
9,9,huobi,2023-04-25T01:21:36.006Z,"{'ask_size': [1.0075, 8.6181, 0.2742, 0.0298, ..."


## <font color= #6B6B6B> <font size = 5> 4. Microstructure Visualizations  </font>

In order to visualize the information obtained from the Book of Orders, a DataFrame was created with the type of exchange market, timestamp, levels, volume of asks, volume of bids, total volume, mid price and VWAP. This created the right structure to then create the time series shown in the following graphs.

#### Exchange 1

In [9]:
series, effective_spreads = dt.consumir_orderbooks(data_files)
binancebtcusdt=series["binance_BTCUSDT"] # it is a dictionary but for visualization terms in the notebook it was downloaded as csv

In [10]:
# grafica exchange 1
vn.grafica_btc(binancebtcusdt)

NameError: name 'px' is not defined

#### Exchange 2

In [ ]:
binanceethusdt=series["binance_ETHUSDT"]
binanceethusdt

In [ ]:
# grafica exchange 2
grafica_eth(binanceethusdt)

#### Exchange 3

In [ ]:
bitfinexbtcusdt=series["bitfinex_BTCUSDT"].head()

In [ ]:
# grafica exchange 3

#### Exchange 4

In [ ]:
bitfinexethusdt=series["bitfinex_ETHUSDT"].head()

In [ ]:
# grafica exchange 4

#### Exchange 5

In [ ]:
huobibtcusdt=series["huobi_BTCUSDT"].head()

In [ ]:
#grafica exchange 5

#### Exchange 6

In [ ]:
huobiethusdt=series["huobi_ETHUSDT"].head()

In [ ]:
# grafica exchange 6

<br>

## <font color= #6B6B6B> <font size = 5> 5. Microstructure Modeling </font>

In order to model the microstructure of the market, the Effective Spread was calculated based on Roll's model (1984). This model provides a simple market microstructure model that allows one to estimate the bid-ask spread from observed transaction prices alone, without information on the underlying bid-ask price quotes and the order. The model developed a simple model to estimate the bid–ask spread using the serial covariance of observable trade prices. The model used in this project is the following:

$$2\sqrt{|COV(\Delta P_t,\Delta P_{t-1}|}$$ 

using the series in $t-5$ for $\Delta P_{t-1}$.

In [ ]:
effective_spreads["binance_BTCUSDT"].head()

In [ ]:
effective_spreads["binance_ETHUSDT"].head()

In [ ]:
effective_spreads["bitfinex_BTCUSDT"].head()

In [ ]:
effective_spreads["bitfinex_ETHUSDT"].head(15)

In [ ]:
effective_spreads["huobi_BTCUSDT"].head(25)

In [ ]:
effective_spreads["huobi_ETHUSDT"].head(15)

# <font color= #6B6B6B> <font size = 5> 6. Conclusions </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

# <font color= #6B6B6B> <font size = 5> 7. References </font>

<hr style="border:0.02in solid gray"> </hr>

Nimalendran, M. (2010). Roll Model. Encyclopedia of Quantitative Finance. https://doi.org/10.1002/9780470061602.eqf18014

Semiparametric Identification of the Bid-Ask Spread in Extended. (2015). Core. https://core.ac.uk/download/pdf/146463242.pdf